# Example Case Study: Advanced MPC Comparison

This notebook is a conversion of the advanced example script `examples/case_22_mpc_comparison.py`. It demonstrates a sophisticated use case for the CHS SDK: comparing the performance of different Model Predictive Control (MPC) strategies on a complex, time-varying system.

This is an advanced tutorial that assumes familiarity with basic MPC concepts.

## 1. Goal of the Simulation

The goal is to control the water level of a system that has **time-varying dynamics**. In the real world, a canal's friction might change as vegetation grows, or a pump's efficiency might decrease over time. Standard controllers with fixed parameters struggle in these situations.

We will compare three MPC strategies:
1.  **Standard MPC**: Uses a fixed, average model of the system. We expect it to perform poorly when the true system parameters drift away from the average.
2.  **Gain-Scheduled MPC**: Uses a predefined "model bank" and switches its internal model based on an observable variable (in this case, the disturbance inflow). This is a simple form of adaptive control.
3.  **Kalman-Adaptive MPC**: The most advanced strategy. It uses an **Extended Kalman Filter (EKF)** to continuously *estimate* the true, changing parameters of the system online and feeds these estimates to the MPC. This allows the controller to adapt to unforeseen changes.

## 2. Imports and Helper Classes

This example requires several advanced components from the SDK. It also defines a custom wrapper class, `ParameterEKFWrapper`, to use the EKF for parameter estimation.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import sys
import os
import collections

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Because the original script had a different path, we use try-except to handle both cases
try:
    # Path in original script
    from water_system_simulator.modeling.integral_plus_delay_model import IntegralPlusDelayModel
    from water_system_simulator.control.mpc_controller import MPCController
    from water_system_simulator.control.gs_mpc_controller import GainScheduledMPCController
    from water_system_simulator.control.kalman_adaptive_mpc_controller import KalmanAdaptiveMPCController
    from water_system_simulator.control.data_assimilation import ExtendedKalmanFilter
    from water_system_simulator.modeling.base_model import BaseModel
except ImportError:
    # Path in our refactored SDK
    from water_system_sdk.src.chs_sdk.modules.modeling.integral_plus_delay_model import IntegralPlusDelayModel
    from water_system_sdk.src.chs_sdk.modules.control.mpc_controller import MPCController
    from water_system_sdk.src.chs_sdk.modules.control.gs_mpc_controller import GainScheduledMPCController
    from water_system_sdk.src.chs_sdk.modules.control.kalman_adaptive_mpc_controller import KalmanAdaptiveMPCController
    from water_system_sdk.src.chs_sdk.modules.control.data_assimilation import ExtendedKalmanFilter
    from water_system_sdk.src.chs_sdk.modules.modeling.base_model import BaseModel


# --- A Custom Agent for Parameter Estimation using an EKF ---
class ParameterEKFWrapper(BaseModel):
    """
    A wrapper to use the ExtendedKalmanFilter for parameter estimation of an
    IntegralPlusDelayModel (K, T).
    """
    def __init__(self, dt, initial_params, P0, Q, R):
        super().__init__()
        self.dt = dt
        self.state_history = collections.deque(maxlen=int(initial_params['T'] / dt) * 2)
        self.input_history = collections.deque(maxlen=int(initial_params['T'] / dt) * 2)

        x0 = np.array([initial_params['K'], initial_params['T']])

        def f(x, *args, **kwargs): return x
        def F_jacobian(x, *args, **kwargs): return np.eye(2)

        def h(x):
            K, T = x
            delay_steps = int(round(T / self.dt))
            if len(self.input_history) > delay_steps:
                y_prev = self.state_history[-1]
                u_delayed = self.input_history[-(delay_steps + 1)]
                return y_prev + K * u_delayed * self.dt
            return self.state_history[-1] if self.state_history else 0.0

        def H_jacobian(x):
            K, T = x
            delay_steps = int(round(T / self.dt))
            if len(self.input_history) > delay_steps + 1:
                u_delayed = self.input_history[-(delay_steps + 1)]
                u_delayed_prev = self.input_history[-(delay_steps + 2)]
                dh_dK = u_delayed * self.dt
                dh_dT = -K * (u_delayed - u_delayed_prev)
                return np.array([[dh_dK, dh_dT]])
            return np.array([[0.0, 0.0]])

        self.ekf = ExtendedKalmanFilter(f, h, F_jacobian, H_jacobian, Q, R, x0, P0)
        self.estimated_params = {'K': x0[0], 'T': x0[1]}

    def step(self, plant_input, plant_output):
        if plant_input is None or plant_output is None: return self.estimated_params
        self.input_history.append(plant_input)
        self.state_history.append(plant_output)
        if len(self.state_history) < 2: return self.estimated_params
        self.ekf.predict()
        self.ekf.update(np.array([plant_output]))
        est_K, est_T = self.ekf.get_state()
        self.estimated_params['K'] = np.clip(est_K, 0.001, 0.02)
        self.estimated_params['T'] = np.clip(est_T, 1000, 3000)
        return self.estimated_params

    def get_state(self): return self.estimated_params

## 3. The Simulation Scenario

The scenario involves:
- A **plant** (an `IntegralPlusDelayModel`) whose `K` and `T` parameters change sinusoidally over time.
- A **disturbance** inflow that steps up halfway through the simulation.
- A manual simulation loop to run each of the three MPC controllers against this challenging plant.

In [ ]:
# This function runs a single simulation for one controller type
def run_manual_simulation(plant, controller, disturbance_ts, true_K_ts, true_T_ts, num_steps, dt, estimator=None):
    history = []
    plant_output = plant.get_state()['output']
    control_action = 0.0

    for i in range(num_steps):
        # Update plant's true parameters
        plant.K = true_K_ts[i]
        plant.T = true_T_ts[i]
        plant.delay_steps = int(round(plant.T / dt))

        disturbance = disturbance_ts[i]

        # Controller step
        controller_args = {
            'current_state': plant_output,
            'disturbance_forecast': np.full(controller.P, disturbance)
        }
        if isinstance(controller, GainScheduledMPCController):
            controller_args['scheduling_variable'] = disturbance
        elif isinstance(controller, KalmanAdaptiveMPCController) and estimator:
             # Estimator step
            total_plant_inflow = disturbance + control_action
            estimated_params = estimator.step(total_plant_inflow, plant_output)
            controller_args['updated_model_params'] = estimated_params

        control_action = controller.step(**controller_args)

        # Plant step
        plant.input.inflow = disturbance
        plant.input.control_inflow = control_action
        plant.step()
        plant_output = plant.get_state()['output']

        # Log data
        log_entry = {
            'time': i * dt,
            'plant_output': plant_output,
            'control_action': control_action,
            'disturbance': disturbance,
            'true_K': plant.K,
            'true_T': plant.T
        }
        if estimator:
            log_entry.update({f'est_{k}': v for k, v in estimator.get_state().items()})
        history.append(log_entry)

    return pd.DataFrame(history)

In [ ]:
print("--- Setting up MPC Comparison Scenario ---")
dt, sim_duration = 100.0, 100000.0
num_steps = int(sim_duration / dt)
set_point = 10.0

time_vector = np.arange(0, sim_duration, dt)
true_initial_K, true_initial_T = 0.008, 1800.0
# True parameters vary sinusoidally over the simulation
true_K_ts = true_initial_K * (1 + 0.25 * np.sin(2 * np.pi * time_vector / sim_duration))
true_T_ts = true_initial_T * (1 - 0.25 * np.sin(2 * np.pi * time_vector / (sim_duration / 2)))

# A step change in disturbance halfway through
disturbance_ts = np.full(num_steps, 50.0)
disturbance_ts[int(num_steps / 2):] = 80.0

P, M = 30, 5
avg_K, avg_T = np.mean(true_K_ts), np.mean(true_T_ts)

# --- Run Scenarios ---
results = {}
for scenario in ["StandardMPC", "GainScheduledMPC", "KalmanAdaptiveMPC"]:
    print(f"--- Running Scenario: {scenario} ---")
    plant = IntegralPlusDelayModel(K=true_initial_K, T=true_initial_T, dt=dt, initial_value=set_point)
    plant.input_buffer = collections.deque([50.0] * int(plant.T / dt), maxlen=int(plant.T / dt))

    controller = None
    estimator = None
    if scenario == "StandardMPC":
        controller = MPCController(prediction_model=IntegralPlusDelayModel(K=avg_K, T=avg_T, dt=dt),
                                 prediction_horizon=P, control_horizon=M, set_point=set_point,
                                 u_min=0, u_max=100, q_weight=1.0, r_weight=0.05)
    elif scenario == "GainScheduledMPC":
        model_bank = [{"threshold": 60, "K": 0.007, "T": 2200}, {"threshold": 90, "K": 0.009, "T": 1600}]
        controller = GainScheduledMPCController(model_bank=model_bank, initial_model_params={'K': avg_K, 'T': avg_T}, dt=dt,
                                              prediction_horizon=P, control_horizon=M, set_point=set_point,
                                              u_min=0, u_max=100, q_weight=1.0, r_weight=0.05)
    elif scenario == "KalmanAdaptiveMPC":
        initial_params_guess = {'K': 0.006, 'T': 2300.0}
        controller = KalmanAdaptiveMPCController(initial_model_params=initial_params_guess, dt=dt,
                                                 prediction_horizon=P, control_horizon=M, set_point=set_point,
                                                 u_min=0, u_max=100, q_weight=1.0, r_weight=0.05)
        P0 = np.diag([0.005**2, 500**2])
        Q = np.diag([(0.008*0.01)**2, (1800*0.01)**2])
        R = np.diag([0.01**2])
        estimator = ParameterEKFWrapper(dt, initial_params_guess, P0, Q, R)

    results[scenario] = run_manual_simulation(plant, controller, disturbance_ts, true_K_ts, true_T_ts, num_steps, dt, estimator)

## 4. Visualization and Analysis

In [ ]:
print("--- Plotting Results ---")
fig, axes = plt.subplots(2, 1, figsize=(15, 12), sharex=True)

# Plot 1: Controller Performance
axes[0].axhline(y=set_point, color='k', linestyle='--', label='Setpoint')
axes[0].plot(results["StandardMPC"]['time'], results["StandardMPC"]['plant_output'], label='Standard MPC')
axes[0].plot(results["GainScheduledMPC"]['time'], results["GainScheduledMPC"]['plant_output'], label='Gain-Scheduled MPC')
axes[0].plot(results["KalmanAdaptiveMPC"]['time'], results["KalmanAdaptiveMPC"]['plant_output'], label='Kalman-Adaptive MPC', linewidth=2)
axes[0].set_ylabel('Water Level (m)')
axes[0].set_title('MPC Controller Performance Comparison')
axes[0].legend()
axes[0].grid(True)

ax0_twin = axes[0].twinx()
ax0_twin.plot(results["StandardMPC"]['time'], results["StandardMPC"]['disturbance'], 'r--', alpha=0.5, label='Disturbance Inflow')
ax0_twin.set_ylabel('Disturbance (m³/s)')
ax0_twin.legend(loc='lower right')

# Plot 2: Kalman Filter Parameter Tracking
kalman_results = results["KalmanAdaptiveMPC"]
axes[1].plot(kalman_results['time'], kalman_results['true_K'], 'b-', label='True K')
axes[1].plot(kalman_results['time'], kalman_results['est_K'], 'c--', label='Estimated K')
axes[1].set_ylabel('Parameter K')
axes[1].legend(loc='upper left')
axes[1].grid(True)

ax1_twin = axes[1].twinx()
ax1_twin.plot(kalman_results['time'], kalman_results['true_T'], 'r-', label='True T')
ax1_twin.plot(kalman_results['time'], kalman_results['est_T'], 'm--', label='Estimated T')
ax1_twin.set_ylabel('Parameter T (s)')
ax1_twin.legend(loc='upper right')

axes[1].set_xlabel('Time (s)')
axes[1].set_title('Kalman Filter Parameter Tracking')

plt.tight_layout()
plt.show()

### Analysis of Results

- **Controller Performance (Top Plot)**: The **Standard MPC** struggles to maintain the setpoint, especially after the disturbance changes, because its internal model is wrong. The **Gain-Scheduled MPC** does better as it can switch models. The **Kalman-Adaptive MPC** (in bold) performs the best, maintaining the tightest control around the setpoint because it is continuously learning the true system dynamics.

- **Parameter Tracking (Bottom Plot)**: This plot shows *why* the adaptive MPC works so well. The dashed lines (the EKF's estimates of K and T) closely follow the solid lines (the true values of K and T). The filter successfully tracks the changing parameters of the system in real-time.

This case study demonstrates the power of combining advanced control (MPC) with online system identification (EKF) to create highly robust and adaptive control systems, a key capability of the CHS platform.